In [7]:
import pandas as pd
import numpy as np
import pandas as pd
import openpyexcel
from openpyexcel import workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyexcel.utils import get_column_letter
from itertools import islice
from os import listdir
import re
import csv
import datetime
import pickle

In [8]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [9]:
company = pd.read_csv('CSV/company_ws.csv',index_col=0)
company

,Balance,Income,Cash
ADBE1.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE2.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE3.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE4.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE5.xlsx,Consolidated_Balance_Sheets,Consolidated_Statements_of_Inc,Consolidated_Statements_of_Cas
AME1.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME2.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME3.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME4.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME5.xlsx,Consolidated_Balance_Sheet,Consolidated_Statement_of_Inco,Consolidated_Statement_of_Cash


In [10]:
def clean_table (table):

    df = table.copy()
    
    df.replace(u'\xa0','', regex=True, inplace=True)
    
    # Replace empty string with nan
    df.replace("^$",np.nan, regex=True, inplace=True)
    
    # Remove [n]
    
    for i in range(1,len(df.columns)):
      if df[df.columns[i]].dtype == object:
        df[df.columns[i]].replace("^\[\d\]", np.nan, regex=True, inplace=True)
    
    # Remove footnotes embedded in table
    
    df.drop(df.loc[df[df.columns[0]].str.contains(r"^\[\d?\]$", case=False, regex=True,na=False)].index,inplace=True)
    
    # Replace nan in first column with ''
    
    df[df.columns[0]].replace(np.nan,'',inplace=True)
    
    #Remove blank columns and rows
    
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
        
    if len(df.columns) > 5:
      # Take the first and the last 3 columns
        
      a = df.iloc[:,:1]
      b = df.iloc[:,-3:]
      df = pd.concat([a,b], axis=1)
    
    #Remove comments in brankets eg (Notes 1) and spurious characters
    
    df[df.columns[0]].replace(r"[\s]?\(.*\)", "", regex=True, inplace=True)
    df[df.columns[0]].replace(r"â€™","", regex=True, inplace=True)
    
    # Deal with situation where column labels are of type date time. Extract year and convert to str
 
    col = list(df.columns[0:])
    
    for i in range(0,len(col)):
      if type(col[i]) is datetime.datetime :
        col[i] = col[i].strftime('%Y')
    
    # Set columns to extraxted year which will be of type string
    # If we don't do this, the following if condition will not work.
    
    df.columns = col
    
    # If none of the columns contain a 4 digit number to signify year then take column headings from first row
    
    if df.columns.str.contains(r"\d\d\d\d", case=False, regex=True,na=False).sum() == 0:
        
      df.iloc[0].fillna('', axis=0, inplace=True)
      
      # Deal with the situation where the first row is read in as date time
    
      for i in range(0,len(df.iloc[0])):
        if type(df.iloc[0][i]) is datetime.datetime :
          df.iloc[0][i] = df.iloc[0][i].strftime('%Y')
            
      df.iloc[0] = df.iloc[0].astype('string')
      col = list(df.iloc[0])
    
      # Drop the first row
        
      df.drop(index=df.index[0], axis=0, inplace=True)
        
    col[0] = 'Index'
    print(col)
    
     # Extraxt year if there is a match
    
    for i in range(1,len(col)):
      res = re.search('(20[0-9][0-9])', col[i])

      if res:
        col[i] = res.group(0)
    
    df.columns = col  
    
    # Remove footnotes embedded in table
    
    df.drop(df.loc[df[df.columns[0]].str.contains(r"^\[\d?\]$", case=False, regex=True,na=False)].index,inplace=True)
    
     # Drop columns and headers that are blank
        
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
    
    # Make sure that first column doesn't have any nulls
    
    df['Index'] = df['Index'].fillna('')
    
    # Set index to first column
    
    df.set_index(df.columns[0], inplace=True)
    
    #Replace single quote with nan
    
    df.replace("'", np.nan, inplace=True)
    
    # Show first row as a validation check
    
    print(pd.DataFrame(df.iloc[0]).T)
    
    return df

In [13]:
def get_line_items(t):
 
  heading = []
  line_item = []
    
  for index, row in t.iterrows():
    
    if row.replace('', np.nan).isna().all():
      # Blank row, so assume heading
    
      heading.append(index)
    else:    
      line_item.append(index)

  return {'heading' : heading,
          'line_item' : line_item}

In [14]:
def get_spreadsheet(company,ticker,file, type):
    
  rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

  dirname = rootdir + ticker
  path = dirname + '/' + file

  xl_file = pd.ExcelFile(path)
  table = xl_file.parse(company.loc[file,type],parse_dates=False)

  return table

In [48]:
table = get_spreadsheet(company,'LVS','LVS1.xlsx', 'Cash')
table

,CONSOLIDATED STATEMENTS OF CASH FLOWS $ in Millions,12 Months Ended,Unnamed: 2,Unnamed: 3
0,NaN,"Dec. 31, 2022 USD ($)","Dec. 31, 2021 USD ($)","Dec. 31, 2020 USD ($)"
1,Cash flows from operating activities from continuing operations:,,,
2,Net loss from continuing operations,-1541,-1469,-1900
3,Adjustments to reconcile net loss to net cash used in operating activities:,,,
4,Depreciation and amortization,1036,1041,997
5,Amortization of leasehold interests in land,55,56,55
6,Amortization of deferred financing costs and original issue discount,57,52,43
7,Change in fair value of derivative asset/liability,1,-1,0
8,Paid-in-kind interest income,-15,0,0
9,Loss on modification or early retirement of debt,0,137,0


In [49]:
k = clean_table(table)
k

['Index', 'Dec. 31, 2022 USD ($)', 'Dec. 31, 2021 USD ($)', 'Dec. 31, 2020 USD ($)']
                                                                 2022 2021  \
Cash flows from operating activities from continuing operations:  NaN  NaN   

                                                                 2020  
Cash flows from operating activities from continuing operations:  NaN  


,2022,2021,2020
Index,,,
Cash flows from operating activities from continuing operations:,NaN,NaN,NaN
Net loss from continuing operations,-1541,-1469,-1900
Adjustments to reconcile net loss to net cash used in operating activities:,NaN,NaN,NaN
Depreciation and amortization,1036,1041,997
Amortization of leasehold interests in land,55,56,55
Amortization of deferred financing costs and original issue discount,57,52,43
Change in fair value of derivative asset/liability,1,-1,0
Paid-in-kind interest income,-15,0,0
Loss on modification or early retirement of debt,0,137,0


In [46]:
rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

headings = []
line_items = []
cash_sheets = {}

for d in listdir(rootdir):
    
  sheets = []
  dirname = rootdir + d
  for f in listdir(dirname):
    if f.endswith('xlsx'):
      print(f)
      table = get_spreadsheet(company,d,f,'Cash')
      t = clean_table(table)
      sheets.append(t)
        
      l_dict = get_line_items(t)      
      headings = headings + l_dict['heading']
      line_items = line_items + l_dict['line_item']
  cash_sheets[d] = sheets

# Remove duplicates from headings and line items

unique_list = set(headings)
headings = list(unique_list)

unique_list = set(line_items)
line_items = list(unique_list)

table_labels = {'headings' : headings,
                'line_items' : line_items}


ADBE1.xlsx
['Index', 'Dec. 02, 2022', 'Dec. 03, 2021', 'Nov. 27, 2020']
                                      2022 2021 2020
Cash flows from operating activities:  NaN  NaN  NaN
ADBE2.xlsx
['Index', 'Nov. 27, 2020', 'Nov. 29, 2019', 'Nov. 30, 2018']
                                      2020 2019 2018
Cash flows from operating activities:  NaN  NaN  NaN
ADBE3.xlsx
['Index', 'Nov. 30, 2018', 'Dec. 01, 2017', 'Dec. 02, 2016']
                                      2018 2017 2016
Cash flows from operating activities:  NaN  NaN  NaN
ADBE4.xlsx
['Index', 'Dec. 02, 2016', 'Nov. 27, 2015', 'Nov. 28, 2014']
                                      2016 2015 2014
Cash flows from operating activities:  NaN  NaN  NaN
ADBE5.xlsx
['Index', 'Nov. 28, 2014', 'Nov. 29, 2013', 'Nov. 30, 2012']
                                      2014 2013 2012
Cash flows from operating activities:  NaN  NaN  NaN
ADM1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 31, 2021', 'Dec. 31, 2020']
                     2022 2021 2020
Ope

C:\Users\priye\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


['Index', 'Dec. 31, 2014', 'Dec. 31, 2013', 'Jun. 30, 2012']
                     2014 2013 2012
Operating Activities  NaN  NaN  NaN
AME1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 31, 2021', 'Dec. 31, 2020']
                      2022 2021 2020
Operating activities:  NaN  NaN  NaN
AME2.xlsx
['Index', 'Dec. 31, 2020', 'Dec. 31, 2019', 'Dec. 31, 2018']
                      2020 2019 2018
Operating activities:  NaN  NaN  NaN
AME3.xlsx
['Index', 'Dec. 31, 2018', 'Dec. 31, 2017', 'Dec. 31, 2016']
                      2018 2017 2016
Operating activities:  NaN  NaN  NaN
AME4.xlsx
['Index', 'Dec. 31, 2016', 'Dec. 31, 2015', 'Dec. 31, 2014']
                      2016 2015 2014
Operating activities:  NaN  NaN  NaN
AME5.xlsx
['Index', 'Dec. 31, 2014', 'Dec. 31, 2013', 'Dec. 31, 2012']
                      2014 2013 2012
Operating activities:  NaN  NaN  NaN
AMGN1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 31, 2021', 'Dec. 31, 2020']
                                      2022 2021 2020
Cash flows from o

                                      2014  2013  2012
CASH FLOWS FROM OPERATING ACTIVITIES   NaN   NaN   NaN
CPB1.xlsx
['Index', 'Jul. 31, 2022', 'Aug. 01, 2021', 'Aug. 02, 2020']
                                      2022 2021 2020
Cash flows from operating activities:  NaN  NaN  NaN
CPB2.xlsx
['Index', 'Jul. 28, 2019', 'Jul. 29, 2018', 'Jul. 30, 2017']
                                      2019 2018 2017
Cash flows from operating activities:  NaN  NaN  NaN
CPB3.xlsx
['Index', 'Jul. 31, 2016', 'Aug. 02, 2015', 'Aug. 03, 2014']
                                      2016 2015 2014
Cash flows from operating activities:  NaN  NaN  NaN
CSCO1.xlsx
['Index', 'Jul. 30, 2022', 'Jul. 31, 2021', 'Jul. 25, 2020']
                                      2022 2021 2020
Cash flows from operating activities:  NaN  NaN  NaN
CSCO2.xlsx
['Index', 'Jul. 25, 2020', 'Jul. 27, 2019', 'Jul. 28, 2018']
                                      2020 2019 2018
Cash flows from operating activities:  NaN  NaN  NaN
CSC

['Index', 'Feb. 03, 2018', 'Jan. 28, 2017', 'Jan. 30, 2016']
                                      2018 2017 2016
Cash Flows from Operating Activities:  NaN  NaN  NaN
KR4.xlsx
['Index', 'Feb. 01, 2014', 'Feb. 02, 2013', 'Jan. 28, 2012']
                                       2014  2013  2012
Cash Flows From Operating Activities:   NaN   NaN   NaN
LNT1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 31, 2021', 'Dec. 31, 2020']
                                      2022 2021 2020
Cash flows from operating activities:  NaN  NaN  NaN
LNT2.xlsx
['Index', 'Dec. 31, 2020', 'Dec. 31, 2019', 'Dec. 31, 2018']
                                      2020 2019 2018
Cash flows from operating activities:  NaN  NaN  NaN
LNT3.xlsx
['Index', 'Dec. 31, 2018', 'Dec. 31, 2017', 'Dec. 31, 2016']
                                      2018 2017 2016
Cash flows from operating activities:  NaN  NaN  NaN
LNT4.xlsx
['Index', 'Dec. 31, 2016', 'Dec. 31, 2015', 'Dec. 31, 2014']
                                      2016 2015 2

['Index', 'Dec. 31, 2014', 'Dec. 31, 2013', 'Dec. 31, 2012']
                                     2014 2013 2012
CASH FLOWS FROM OPERATING ACTIVITIES  NaN  NaN  NaN
Pepsico1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 25, 2021', 'Dec. 26, 2020']
                     2022 2021 2020
Operating Activities  NaN  NaN  NaN
Pepsico2.xlsx
['Index', 'Dec. 28, 2019', 'Dec. 29, 2018', 'Dec. 30, 2017']
                     2019 2018 2017
Operating Activities  NaN  NaN  NaN
Pepsico3.xlsx
['Index', 'Dec. 26, 2015', 'Dec. 27, 2014', 'Dec. 28, 2013']
                                   2015 2014 2013
Statement of Cash Flows [Abstract]  NaN  NaN  NaN
REGN1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 31, 2021', 'Dec. 31, 2020']
                                      2022 2021 2020
Cash flows from operating activities:  NaN  NaN  NaN
REGN2.xlsx
['Index', 'Dec. 31, 2020', 'Dec. 31, 2019', 'Dec. 31, 2018']
                                      2020 2019 2018
Cash flows from operating activities:  NaN  NaN  NaN
REGN3.xlsx

Statement of Cash Flows [Abstract]  NaN  NaN  NaN
WAB5.xlsx
['Index', 'Dec. 31, 2014', 'Dec. 31, 2013', 'Dec. 31, 2012']
                                   2014 2013 2012
Statement Of Cash Flows [Abstract]  NaN  NaN  NaN
WEC1.xlsx
['Index', 'Dec. 31, 2022', 'Dec. 31, 2021', 'Dec. 31, 2020']
                     2022 2021 2020
Operating activities  NaN  NaN  NaN
WEC2.xlsx
['Index', 'Dec. 31, 2020', 'Dec. 31, 2019', 'Dec. 31, 2018']
                     2020 2019 2018
Operating activities  NaN  NaN  NaN
WEC3.xlsx
['Index', 'Dec. 31, 2018', 'Dec. 31, 2017', 'Dec. 31, 2016']
                     2018 2017 2016
Operating activities  NaN  NaN  NaN
WEC4.xlsx
['Index', 'Dec. 31, 2016', 'Dec. 31, 2015', 'Dec. 31, 2014']
                     2016 2015 2014
Operating activities  NaN  NaN  NaN
WEC5.xlsx
['Index', 'Dec. 31, 2014', 'Dec. 31, 2013', 'Dec. 31, 2012']
                     2014 2013 2012
Operating Activities  NaN  NaN  NaN
WMT1.xlsx
['Index', 'Jan. 31, 2023', 'Jan. 31, 2022', 'Jan. 31, 

In [50]:
def save_line_items(table_labels):
    
  with open('CSV/cash_headings.csv', 'w',encoding='utf-8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(table_labels['headings'])
    
  with open('CSV/cash_line_items.csv', 'w',encoding='utf-8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(table_labels['line_items'])


In [51]:
def save_table_labels(table_labels):

  with open('cash_table_labels.pkl', 'wb') as fp:
    pickle.dump(table_labels, fp)


In [52]:
def save_cash_sheets(cash_sheets):
  with open('cash_sheets.pkl', 'wb') as fp:
    pickle.dump(cash_sheets, fp)

In [53]:
save_line_items(table_labels)

In [54]:
save_table_labels(table_labels)

In [55]:
save_cash_sheets(cash_sheets)

In [29]:
# Check for index values defined as tuples with nan

for b in l:
    #print(type(b))
    if type(b) is tuple:
        print(b)